In [1]:
from token_shap import TokenSHAP

In [2]:
model_name = "llama3"
tokenizer_path ="NousResearch/Hermes-2-Theta-Llama-3-8B"
tshap = TokenSHAP(model_name, tokenizer_path)

/opt/anaconda3/envs/opensora/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import json
import os

# Initialize lists to store results
sampling_ratios = [0.8, 0.6, 0.4, 0.2, 0.0]
num_questions = 10
similarities = {ratio: [] for ratio in sampling_ratios}

# Define the questions
questions = [
    "Can you describe to me how deep learning might affect the job market in the future?",
    "What are the ethical implications of artificial intelligence in healthcare?",
    "How can machine learning be used to improve cybersecurity?",
    "What is the impact of AI on the education sector?",
    "How is deep learning changing the landscape of autonomous vehicles?",
    "What are the environmental benefits of using AI in energy management?",
    "How can AI help in personalized medicine?",
    "What are the challenges of implementing AI in financial services?",
    "How does AI influence modern marketing strategies?",
    "What are the future prospects of AI in robotics?"
]

def shapley_dict_to_vector(shapley_dict):
    return np.array(list(shapley_dict.values()))

output_file = 'similarity_results.json'
if os.path.exists(output_file):
    with open(output_file, 'r') as f:
        similarities = json.load(f)
else:
    similarities = {ratio: [] for ratio in sampling_ratios}

for question in questions:
    # Calculate baseline Shapley values
    baseline_shap = tshap.analyze(question, sampling_ratio=1.0, splitter=' ')
    baseline_shap_values = shapley_dict_to_vector(tshap.shapley_values)
    
    for ratio in sampling_ratios:
        # Calculate Shapley values for the given sampling ratio
        sampled_shap = tshap.analyze(question, sampling_ratio=ratio, splitter=' ')
        sampled_shap_values = shapley_dict_to_vector(tshap.shapley_values)
        
        # Calculate similarity between baseline and sampled Shapley values
        similarity = cosine_similarity(
            [baseline_shap_values], 
            [sampled_shap_values]
        )[0, 0]
        
        # Store the similarity
        similarities[ratio].append(similarity)
        
        # Save the results to disk
        with open(output_file, 'w') as f:
            json.dump(similarities, f)

# Calculate average similarities for each sampling ratio
average_similarities = {ratio: np.mean(similarities[ratio]) for ratio in sampling_ratios}

# Plot the results
plt.figure(figsize=(10, 6))
plt.plot(sampling_ratios, [average_similarities[ratio] for ratio in sampling_ratios], marker='o')
plt.xlabel('Sampling Ratio')
plt.ylabel('Average Cosine Similarity')
plt.title('Average Similarity of Shapley Values at Different Sampling Ratios')
plt.grid(True)
plt.show()